<a href="https://colab.research.google.com/github/Simo56a0/machineLEARNING/blob/master/capsnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Update dataset paths
BASE_PATH = "/content/drive/MyDrive/DATASET_ug_sign_language/"
VIDEO_PATH = "/content/drive/MyDrive/DATASET_ug_sign_language/videos/"
FRAMES_PATH = "/content/drive/MyDrive/DATASET_ug_sign_language/frames1"
ANNOTATIONS_PATH = "/content/drive/MyDrive/DATASET_ug_sign_language/encoded_sign_labels.csv"


In [ ]:
!pip install opencv-python tensorflow-addons


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
import tensorflow as tf
print("GPU Available: ", tf.config.list_physical_devices('GPU'))


GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import os
# Verify dataset
print("Files in Frames Folder: ", os.listdir(FRAMES_PATH)[:10])
print("Files in Videos Folder: ", os.listdir(VIDEO_PATH)[:10])


Files in Frames Folder:  ['0060.mp4_frame_32.jpg', '0060.mp4_frame_33.jpg', '0061.mp4_frame_0.jpg', '0061.mp4_frame_1.jpg', '0061.mp4_frame_2.jpg', '0061.mp4_frame_3.jpg', '0061.mp4_frame_4.jpg', '0061.mp4_frame_5.jpg', '0061.mp4_frame_6.jpg', '0061.mp4_frame_7.jpg']
Files in Videos Folder:  ['0066.mp4', '0068.mp4', '0067.mp4', '0069.mp4', '0070.mp4', '0071.mp4', '0072.mp4', '0073.mp4', '0074.mp4', '0075.mp4']


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

def load_data(frames_path, df, img_size=(64, 64)):
    X, Y = [], []
    missing_files = 0

    for idx, row in df.iterrows():
        label = row['encoded_label']
        video_id = row['video_ID']

        # Look for multiple frames for a given video
        frame_files = [f for f in os.listdir(frames_path) if f.startswith(f"{video_id}_frame_")]

        if frame_files:
            for frame_file in frame_files:
                frame_path = os.path.join(frames_path, frame_file)

                img = cv2.imread(frame_path)
                if img is None:
                    missing_files += 1
                    print(f"Corrupt file: {frame_path}")  # Debugging
                    continue

                img = cv2.resize(img, img_size)
                img = img / 255.0  # Normalize
                X.append(img)
                Y.append(label)
        else:
            missing_files += 1
            print(f"Missing frames for video: {video_id}")

    print(f"Total Missing or Corrupt Files: {missing_files}")  # Debugging
    return np.array(X), np.array(Y)

df = pd.read_csv(ANNOTATIONS_PATH)
X, Y = load_data(FRAMES_PATH, df)

print(f"Dataset shape: {X.shape}, Labels: {Y.shape}")


Total Missing or Corrupt Files: 0
Dataset shape: (4247, 64, 64, 3), Labels: (4247,)


In [ ]:
import pandas as pd

# Load annotations
df = pd.read_csv(ANNOTATIONS_PATH)

print("Dataset Shape:", df.shape)
print("First 5 rows:")
print(df.head())

# Check for missing values
print("Missing Values:", df.isnull().sum())


Dataset Shape: (90, 2)
First 5 rows:
   video_ID sign_word
0  0001.mp4     hello
1  0002.mp4       you
2  0003.mp4       how
3  0004.mp4      name
4  0005.mp4      what
Missing Values: video_ID     0
sign_word    0
dtype: int64


In [ ]:
# Use stratified splitting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)

print(f"Training: {X_train.shape}, Testing: {X_test.shape}")


Training: (3397, 64, 64, 3), Testing: (850, 64, 64, 3)


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

datagen.fit(X_train)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Reshape, Lambda, Dense, Input, MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

# Squash function (with epsilon for stability)
def squash(vectors, axis=-1):
    epsilon = 1e-7  # Increase epsilon for better stability
    s_squared_norm = tf.reduce_sum(tf.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm + epsilon)
    return scale * vectors / (tf.sqrt(s_squared_norm + epsilon))

# Updated CapsuleLayer with routing logits clipping
class CapsuleLayer(tf.keras.layers.Layer):
    def __init__(self, num_capsules, dim_capsule, num_routing=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsule = dim_capsule
        self.num_routing = num_routing

    def build(self, input_shape):
        # input_shape: (batch_size, input_capsules, input_dim)
        self.input_capsules = input_shape[1]
        self.input_dim = input_shape[2]
        # Weight matrix: shape (input_capsules, num_capsules, dim_capsule, input_dim)
        self.W = self.add_weight(
            shape=(self.input_capsules, self.num_capsules, self.dim_capsule, self.input_dim),
            initializer='glorot_uniform',
            trainable=True,
            name='W'
        )
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs):
        # inputs: (batch_size, input_capsules, input_dim)
        inputs_expanded = tf.expand_dims(inputs, 2)  # (batch_size, input_capsules, 1, input_dim)
        inputs_expanded = tf.expand_dims(inputs_expanded, -1)  # (batch_size, input_capsules, 1, input_dim, 1)
        inputs_tiled = tf.tile(inputs_expanded, [1, 1, self.num_capsules, 1, 1])  # (batch_size, input_capsules, num_capsules, input_dim, 1)

        # Expand W: shape becomes (1, input_capsules, num_capsules, dim_capsule, input_dim)
        W_tiled = tf.expand_dims(self.W, 0)
        # Compute u_hat via matrix multiplication: result shape (batch_size, input_capsules, num_capsules, dim_capsule, 1)
        u_hat = tf.matmul(W_tiled, inputs_tiled)
        u_hat = tf.squeeze(u_hat, axis=-1)  # (batch_size, input_capsules, num_capsules, dim_capsule)

        # Initialize routing logits b to zeros.
        batch_size = tf.shape(inputs)[0]
        b = tf.zeros(shape=(batch_size, self.input_capsules, self.num_capsules))

        # Routing algorithm
        for i in range(self.num_routing):
            # Clip b to prevent extremely high or low values, then compute softmax
            c = tf.nn.softmax(tf.clip_by_value(b, -10.0, 10.0), axis=2)  # (batch_size, input_capsules, num_capsules)
            c_expanded = tf.expand_dims(c, -1)  # (batch_size, input_capsules, num_capsules, 1)
            s = tf.reduce_sum(c_expanded * u_hat, axis=1)  # (batch_size, num_capsules, dim_capsule)
            v = squash(s)
            if i < self.num_routing - 1:
                v_expanded = tf.expand_dims(v, 1)  # (batch_size, 1, num_capsules, dim_capsule)
                b += tf.reduce_sum(u_hat * v_expanded, axis=-1)
        return v  # (batch_size, num_capsules, dim_capsule)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules, self.dim_capsule)

# Build the Capsule Network model with an added pooling layer to reduce memory consumption
def build_capsule_network(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # First Convolutional Layer
    conv1 = Conv2D(256, (9, 9), strides=1, padding='valid')(inputs)
    conv1 = tf.keras.layers.BatchNormalization()(conv1)
    conv1 = tf.keras.layers.Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Primary Capsules: Convolutional layer followed by reshaping
    primary_caps = Conv2D(32 * 8, (9, 9), strides=2, padding='valid')(pool1)
    primary_caps = tf.keras.layers.BatchNormalization()(primary_caps)
    primary_caps = tf.keras.layers.Activation('relu')(primary_caps)
    primary_caps = Reshape((-1, 8))(primary_caps)
    primary_caps = Lambda(squash)(primary_caps)

    # Capsule Layer: one capsule per class
    digit_caps = CapsuleLayer(num_classes, 16, num_routing=3)(primary_caps)

    # Compute the length of each capsule vector to get class probabilities
    outputs = Lambda(lambda z: tf.sqrt(tf.reduce_sum(tf.square(z), axis=-1)))(digit_caps)

    model = Model(inputs, outputs)
    return model

# Example usage:
input_shape = (64, 64, 3)
num_classes = len(df['encoded_label'].unique())  # Update according to your CSV
model = build_capsule_network(input_shape, num_classes)

# Use a lower learning rate and gradient clipping to enhance stability
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=0.5)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

# Then train with a reduced batch size if needed:
# history = model.fit(datagen.flow(X_train, Y_train, batch_size=16),
#                     validation_data=(X_test, Y_test),
#                     epochs=50,
#                     callbacks=[early_stopping])


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 64, 64, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 56, 56, 256)         │          62,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 56, 56, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 56, 56, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 10, 10, 256)         │       5,308,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 10, 10, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 10, 10, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_3 (Reshape)                  │ (None, 3200, 8)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_6 (Lambda)                    │ (None, 3200, 8)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ capsule_layer_3 (CapsuleLayer)       │ (None, 85, 16)              │      34,816,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_7 (Lambda)                    │ (None, 85)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40,189,184 (153.31 MB)

 Trainable params: 40,188,160 (153.31 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [ ]:
history = model.fit(
    datagen.flow(X_train, Y_train, batch_size=16),  # reduced batch size
    validation_data=(X_test, Y_test),
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - accuracy: 0.0478 - loss: 4.2220 - val_accuracy: 0.0447 - val_loss: 4.2083
Epoch 2/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 21s 96ms/step - accuracy: 0.0399 - loss: 4.2283 - val_accuracy: 0.0447 - val_loss: 4.2107
Epoch 3/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - accuracy: 0.0431 - loss: 4.2169 - val_accuracy: 0.0447 - val_loss: 4.1977
Epoch 4/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 21s 96ms/step - accuracy: 0.0487 - loss: 4.2125 - val_accuracy: 0.0447 - val_loss: 4.1940
Epoch 5/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 20s 94ms/step - accuracy: 0.0465 - loss: 4.2093 - val_accuracy: 0.0447 - val_loss: 4.1992
Epoch 6/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 21s 96ms/step - accuracy: 0.0411 - loss: 4.1998 - val_accuracy: 0.0447 - val_loss: 4.1893
Epoch 7/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 20s 96ms/step - accuracy: 0.0444 - loss: 4.2057 - val_accuracy: 0.0447 - val_loss: 4.1847
Epoch 8/20
213/213 ━━━━━━━━━━━━━━━━━━━━ 20s 96ms/step - accuracy: 0.0398 - loss: 4.2065 - 

In [ ]:
# Plot Accuracy & Loss
plt.figure(figsize=(12, 5))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

# Evaluate Model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {test_acc:.4f}")
